# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(131)
initial_params = np.random.random([46])

INITIALIZATION_METHOD = 'Amplitude'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [4]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label in [0, 1, 2, 3]:
        x.append(example)
        y.append(-1)
    else:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=3)

In [9]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(3), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(3), rotation='Y')

    # First layer
    ArbitraryUnitary(params[:15], wires=[1, 2])

    # Second layer
    ArbitraryUnitary(params[15:30], wires=[0, 1])

    # Third layer
    ArbitraryUnitary(params[30:45], wires=[1, 2])

    # Fourth layer
    qml.RY(params[45], wires=2)

    # Measurement
    return qml.expval(qml.PauliZ(2))

## Circuit example

In [10]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.65015361 0.94810917 0.38802889 0.64129616 0.69051205 0.12660931
 0.23946678 0.25415707 0.42644165 0.83900255 0.74503365 0.38067928
 0.26169292 0.05333379 0.43689638 0.20897912 0.59441102 0.09890353
 0.22409353 0.5842624  0.95908107 0.20988382 0.66133746 0.50261295
 0.32029143 0.12506485 0.80688893 0.98696002 0.54304141 0.23132314
 0.60351254 0.17669598 0.88653747 0.58902228 0.72117264 0.27567029
 0.78811469 0.1326223  0.39971595 0.62982409 0.42404345 0.16187284
 0.52034418 0.6070413  0.5808057  0.82111597]

Example features: [ 1.7989834   0.54691873 -0.57957927  3.65304245 -0.41769116 -0.92091488
  0.67359091 -2.1211999 ]

Expectation value: -0.2108176139475219

 0: ──╭QubitStateVector(M0)──H──RZ(0.209)──H──RX(1.57)──RZ(0.594)──RX(-1.57)──RZ(0.0989)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

# Accuracy test definition

In [11]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [12]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.506842737094838
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.695078031212485
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.7428571428571429
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.7382953181272509
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.7459783913565425
best accuracy so far!
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.7438175270108043
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.7409363745498199
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.7231692677070829
epoch 71
epoch 72
epoch 73
epoch 74
epoch 75
epoch 76

epoch 341
epoch 342
epoch 343
epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.7968787515006003
test_accuracies: [0.7409363745498199, 0.7231692677070829, 0.7546218487394958, 0.7639855942376951, 0.7394957983193278, 0.7923169267707083, 0.7623049219687875, 0.7906362545018008, 0.7723889555822329, 0.7963985594237695, 0.7755102040816326, 0.7579831932773109, 0.7411764705882353, 0.7699879951980793, 0.7680672268907562, 0.7344537815126051, 0.7584633853541416, 0.7853541416566627, 0.7527010804321729, 0.7683073229291717, 0.7997599039615846, 0.7747899159663866, 0.7827130852340937, 0.7771908763505402, 0.7807923169267708, 0.7702280912364946, 0.7623049219687875, 0.7858343337334934, 0.8052821128451381, 0.7968787515006003]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.7971188475390156
test_accuracies: [0.7231692677070829, 0.7546218487394958, 0.7639855942376951, 0.7394957983193278, 0.7923169267707083, 0.7623

epoch 451
epoch 452
epoch 453
epoch 454
epoch 455
epoch 456
epoch 457
epoch 458
epoch 459
epoch 460
accuracy: 0.7563025210084033
test_accuracies: [0.7579831932773109, 0.7411764705882353, 0.7699879951980793, 0.7680672268907562, 0.7344537815126051, 0.7584633853541416, 0.7853541416566627, 0.7527010804321729, 0.7683073229291717, 0.7997599039615846, 0.7747899159663866, 0.7827130852340937, 0.7771908763505402, 0.7807923169267708, 0.7702280912364946, 0.7623049219687875, 0.7858343337334934, 0.8052821128451381, 0.7968787515006003, 0.7971188475390156, 0.7822328931572629, 0.7807923169267708, 0.7680672268907562, 0.7726290516206482, 0.7781512605042017, 0.7719087635054022, 0.778391356542617, 0.7719087635054022, 0.7591836734693878, 0.7563025210084033]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.7791116446578632
test_accuracies: [0.7411764705882353, 0.7699879951980793, 0.7680672268907562, 0.7344537815126051, 0.7584633853541416, 0.78535

epoch 561
epoch 562
epoch 563
epoch 564
epoch 565
epoch 566
epoch 567
epoch 568
epoch 569
epoch 570
accuracy: 0.792076830732293
test_accuracies: [0.7827130852340937, 0.7771908763505402, 0.7807923169267708, 0.7702280912364946, 0.7623049219687875, 0.7858343337334934, 0.8052821128451381, 0.7968787515006003, 0.7971188475390156, 0.7822328931572629, 0.7807923169267708, 0.7680672268907562, 0.7726290516206482, 0.7781512605042017, 0.7719087635054022, 0.778391356542617, 0.7719087635054022, 0.7591836734693878, 0.7563025210084033, 0.7791116446578632, 0.7831932773109244, 0.7831932773109244, 0.7781512605042017, 0.7740696278511404, 0.7723889555822329, 0.7961584633853541, 0.787515006002401, 0.7867947178871548, 0.7846338535414166, 0.792076830732293]
epoch 571
epoch 572
epoch 573
epoch 574
epoch 575
epoch 576
epoch 577
epoch 578
epoch 579
epoch 580
accuracy: 0.7699879951980793
test_accuracies: [0.7771908763505402, 0.7807923169267708, 0.7702280912364946, 0.7623049219687875, 0.7858343337334934, 0.80528211

In [13]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 0.51648937  1.18222715 -0.04482765  0.20858405 -0.09970275 -0.00246735
  0.42713029  0.16514222  0.53584643  1.02179672  0.40345205  0.39107703
  0.07216327  1.00586783  0.75750142 -0.42645637  0.49194629  0.65061388
 -0.35669044  0.10930101  0.79850883 -0.17865363  0.5891947   0.79599452
 -0.100082   -0.38218501  2.03296675  0.34646881  0.53032162  0.68522662
  0.85312535 -0.03727796  1.3355731   0.67054849  0.2332901   0.0190848
  1.06020304  0.52172277  0.34358368  0.2041607  -0.291883    1.20788779
  0.03955215  0.47288902  0.77681968  0.31181293]


# Testing

In [14]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.7906666666666666


In [15]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 11.485447406768799
training time: 9168.842572450638
test time: 346.9376730918884
total time: 9527.265692949295
